In [60]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import pickle
from flask import Flask, request
from flask_restful import Resource, Api
from flask import jsonify

In [61]:
iris=load_iris()
iris.data = np.delete(iris.data, [1,3], 1)
iris.feature_names = iris.feature_names[::2]
iris.data = np.delete(iris.data, np.where(iris.target==2), axis=0)
iris.target = np.delete(iris.target, np.where(iris.target==2), axis=0)

df = pd.DataFrame(data=np.c_[iris['data'],iris['target']],columns=iris['feature_names']+['target'])
df.head()

,sepal length (cm),petal length (cm),target
0,5.1,1.4,0.0
1,4.9,1.4,0.0
2,4.7,1.3,0.0
3,4.6,1.5,0.0
4,5.0,1.4,0.0


In [62]:
class Perceptron:
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.zeros(1 + X.shape[1])
        # zainicjowanie weights
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                update=self.eta*(target-self.predict(xi))
                self.w_[1:] += update *xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]
        # weighted sum
    
    def predict(self, X):
        return np.where(self.net_input(X) >= 0, 1, 0)

In [63]:
model = Perceptron()
X=df.iloc[:,:2].values
y=df.target.values
model.fit(X,y)
with open("model_rta.pkl", "wb") as fh:
    pickle.dump(model, fh)

In [64]:
app = Flask(__name__)

@app.route("/api/predict", methods = ['GET'])
def prediction():

    with open("model_rta.pkl", "rb") as fh:    
        loaded_model = pickle.load(fh)

    sl = float(request.args.get("sl"))
    pl = float(request.args.get("pl"))
    p = loaded_model.predict(np.array([sl, pl]))
    
    if p == 1:
        name =  "versicolor"
    else:
        name = "setosa"
        
    return f"Predicted class for sepal_length = {sl}, and petal_length = {pl} is {name}."


app.run(port='5000') 
# http://127.0.0.1:5000/api/predict?&sl=4.5&pl=3.2

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2022 19:51:00] "GET /api/predict?&sl=4.5&pl=3.2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 19:51:08] "GET /api/predict?&sl=4.5&pl=3.2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 19:51:09] "GET /api/predict?&sl=4.5&pl=3.2 HTTP/1.1" 200 -


In [59]:
import requests
response = requests.get("http://127.0.0.1:5000/api/predict?&sl=4.5&pl=3.2")
print(response.content)


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/predict?&sl=4.5&pl=3.2 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E243643DC0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))